In [1]:
# Imports necesarios
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
import os
from dotenv import load_dotenv

/Users/adrianinfantes/Desktop/AIR/CollegeStudies/AISolutionsArchitechPath/Langchain/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Cargar las claves desde el archivo .env
load_dotenv()

# Configuración de la base de datos
mysql_host = os.getenv("MYSQL_HOST")
mysql_port = os.getenv("MYSQL_PORT")
mysql_user = os.getenv("MYSQL_USER")
mysql_password = os.getenv("MYSQL_PASSWORD")

In [3]:
# Crear la conexión a la base de datos
mysql_db_uri = f"mysql+pymysql://{mysql_user}:{mysql_password}@{mysql_host}:{mysql_port}/usuarios_cibersecurity"
db = SQLDatabase.from_uri(mysql_db_uri)

# Inicializar el modelo de lenguaje
llm = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv("OPENAI_API_KEY"))

In [4]:
# Definir el prompt para generar la consulta SQL
sql_prompt = PromptTemplate.from_template(
    """Based on the table schema below, write only a SQL query (without any markdown formatting or code blocks) that would answer the user's question.

    {table_info}

    Question: {question}
    Remember: Return ONLY the SQL query, without any markdown formatting, explanation, or code blocks.
    SQL Query: """
)

# Definir el prompt para la respuesta final
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question in Spanish.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

In [5]:
def clean_sql_query(query: str) -> str:
    """Limpia la consulta SQL removiendo marcadores de código y espacios innecesarios."""
    # Remover marcadores de código markdown
    query = query.replace("```sql", "").replace("```", "")
    # Limpiar espacios extra y saltos de línea
    query = query.strip()
    return query

# Crear la cadena para generar la consulta SQL
def generate_sql_query(inputs):
    prompt_value = sql_prompt.format_prompt(
        question=inputs["question"],
        table_info=db.get_table_info()
    )
    raw_query = llm.predict(prompt_value.to_string())
    # Limpiar la consulta antes de devolverla
    return clean_sql_query(raw_query)

# Crear la cadena para ejecutar la consulta
def execute_sql_query(inputs):
    query = inputs["query"]
    try:
        # Imprimir la consulta para debugging
        print("\nConsulta SQL a ejecutar:", query)
        return db.run(query)
    except Exception as e:
        print(f"Error ejecutando consulta: {str(e)}")
        raise e

In [6]:
# Crear la cadena de procesamiento completa
chain = (
    RunnablePassthrough.assign(query=generate_sql_query).assign(
        result=execute_sql_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)

In [7]:
# Función principal para procesar preguntas
def get_db_response(question: str) -> str:
    """
    Obtiene una respuesta en lenguaje natural a partir de una pregunta sobre la base de datos.

    Args:
        question (str): Pregunta en lenguaje natural

    Returns:
        str: Respuesta en lenguaje natural
    """
    try:
        # Imprimir información de la base de datos para debugging
        print("Esquema de la base de datos:")
        print(db.get_table_info())

        # Procesar la pregunta
        response = chain.invoke({"question": question})
        return response
    except Exception as e:
        print(f"Error detallado: {str(e)}")
        return f"Error al procesar la pregunta: {str(e)}"

In [8]:
# Ejemplo de uso

# Configurar logging básico
import logging
logging.basicConfig(level=logging.INFO)

# Ejemplo de pregunta
question = "¿Cuáles son los últimos 5 usuarios registrados?"
print("\nPregunta:", question)
result = get_db_response(question)
print("\nRespuesta:", result)


Pregunta: ¿Cuáles son los últimos 5 usuarios registrados?
Esquema de la base de datos:

CREATE TABLE usuarios (
	id_usuario INTEGER NOT NULL AUTO_INCREMENT, 
	nombre VARCHAR(100) NOT NULL, 
	nivel_estudios VARCHAR(50) NOT NULL, 
	especialidad VARCHAR(100), 
	`experiencia_años` INTEGER, 
	email VARCHAR(100), 
	telefono VARCHAR(15), 
	ciudad VARCHAR(50), 
	fecha_registro DATE DEFAULT (curdate()), 
	comentarios TEXT, 
	PRIMARY KEY (id_usuario)
)ENGINE=InnoDB COLLATE utf8mb4_0900_ai_ci DEFAULT CHARSET=utf8mb4

/*
3 rows from usuarios table:
id_usuario	nombre	nivel_estudios	especialidad	experiencia_años	email	telefono	ciudad	fecha_registro	comentarios
1	Juan Pérez	Licenciatura	Seguridad de Redes	5	juan.perez@email.com	123456789	Madrid	2024-11-16	Especialista en firewalls y gestión de redes seguras.
2	María López	Máster	Criptografía	3	maria.lopez@email.com	987654321	Barcelona	2024-11-16	Experta en algoritmos de cifrado y técnicas de protección de datos.
3	Carlos García	Doctorado	Análisis de

/var/folders/rn/1q7mj00d6b3fwpkj8tndfp8c0000gn/T/ipykernel_16618/2424357026.py:15: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  raw_query = llm.predict(prompt_value.to_string())
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Consulta SQL a ejecutar: SELECT * FROM usuarios ORDER BY fecha_registro DESC LIMIT 5;


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Respuesta: Los últimos 5 usuarios registrados son:

1. **Juan Pérez** - Licenciatura en Seguridad de Redes, especialista en firewalls y gestión de redes seguras. (Email: juan.perez@email.com, Ubicación: Madrid)
2. **María López** - Máster en Criptografía, experta en algoritmos de cifrado y técnicas de protección de datos. (Email: maria.lopez@email.com, Ubicación: Barcelona)
3. **Carlos García** - Doctorado en Análisis de Vulnerabilidades, lideró proyectos de investigación sobre vulnerabilidades en sistemas operativos. (Email: carlos.garcia@email.com, Ubicación: Valencia)
4. **Ana Sánchez** - Licenciatura en Ingeniería Social, con enfoque en prevención de ataques de ingeniería social. (Email: ana.sanchez@email.com, Ubicación: Sevilla)
5. **Laura Gómez** - Máster en Forense Digital, perito especializado en análisis forense de dispositivos electrónicos. (Email: laura.gomez@email.com, Ubicación: Bilbao)


In [2]:
# Imports necesarios
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter
import os
from dotenv import load_dotenv
import logging

In [2]:
# Configurar logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [4]:
class DatabaseQueryChain:
    """Clase para manejar consultas a la base de datos usando LangChain."""

    def __init__(self):
        """Inicializar la cadena de consulta."""
        # Cargar variables de entorno
        load_dotenv()

        # Configurar conexión a la base de datos
        self.db = self._setup_database()

        # Configurar el modelo de lenguaje
        self.llm = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0,  # Reducir aleatoriedad para consultas más consistentes
            api_key=os.getenv("OPENAI_API_KEY")
        )

        # Configurar las cadenas de procesamiento
        self._setup_chains()

    def _setup_database(self) -> SQLDatabase:
        """Configurar la conexión a la base de datos."""
        try:
            db_uri = f"mysql+pymysql://{os.getenv('MYSQL_USER')}:{os.getenv('MYSQL_PASSWORD')}@" \
                     f"{os.getenv('MYSQL_HOST')}:{os.getenv('MYSQL_PORT')}/usuarios_cibersecurity"
            return SQLDatabase.from_uri(db_uri)
        except Exception as e:
            logger.error(f"Error conectando a la base de datos: {e}")
            raise

    def _setup_chains(self):
        """Configurar las cadenas de procesamiento."""
        # Prompt para generar consultas SQL
        self.sql_prompt = PromptTemplate.from_template(
            """Given the database schema below, write a clear, efficient SQL query to answer the user's question.
            Return ONLY the SQL query without any markdown, comments, or explanations.

            Schema:
            {table_info}

            Question: {question}
            SQL Query: """
        )

        # Prompt para generar respuestas en lenguaje natural
        self.answer_prompt = PromptTemplate.from_template(
            """Based on the following information, provide a clear and concise answer in Spanish:

            Question: {question}
            SQL Query: {query}
            SQL Result: {result}

            Answer in a user-friendly way, providing context when useful.
            Answer: """
        )

        # Configurar la cadena completa
        self._setup_processing_chain()

    def _setup_processing_chain(self):
        """Configurar la cadena de procesamiento principal."""
        self.chain = (
            RunnablePassthrough.assign(
                query=self._generate_sql_query
            ).assign(
                result=self._execute_sql_query
            )
            | self.answer_prompt
            | self.llm
            | StrOutputParser()
        )

    def _clean_sql_query(self, query: str) -> str:
        """Limpiar y formatear la consulta SQL."""
        # Remover marcadores de código y espacios innecesarios
        query = query.replace("```sql", "").replace("```", "").strip()
        return " ".join(query.split())  # Normalizar espacios

    def _generate_sql_query(self, inputs: dict) -> str:
        """Generar la consulta SQL a partir de la pregunta."""
        try:
            prompt_value = self.sql_prompt.format_prompt(
                question=inputs["question"],
                table_info=self.db.get_table_info()
            )
            query = self.llm.predict(prompt_value.to_string())
            clean_query = self._clean_sql_query(query)
            logger.info(f"Consulta SQL generada: {clean_query}")
            return clean_query
        except Exception as e:
            logger.error(f"Error generando consulta SQL: {e}")
            raise

    def _execute_sql_query(self, inputs: dict) -> str:
        """Ejecutar la consulta SQL."""
        try:
            result = self.db.run(inputs["query"])
            logger.info(f"Resultado de la consulta: {result}")
            return result
        except Exception as e:
            logger.error(f"Error ejecutando consulta SQL: {e}")
            raise

    def process_question(self, question: str) -> str:
        """
        Procesar una pregunta y obtener una respuesta.

        Args:
            question (str): Pregunta en lenguaje natural

        Returns:
            str: Respuesta en lenguaje natural
        """
        try:
            logger.info(f"Procesando pregunta: {question}")
            response = self.chain.invoke({"question": question})
            logger.info(f"Respuesta generada exitosamente")
            return response
        except Exception as e:
            logger.error(f"Error procesando pregunta: {e}")
            return f"Lo siento, hubo un error al procesar tu pregunta: {str(e)}"

In [5]:
"""Función principal para demostración."""
# Crear instancia de la cadena
query_chain = DatabaseQueryChain()

# Ejemplos de preguntas
questions = [
    "¿Cuáles son los últimos 5 usuarios registrados?",
    "¿Cuántos usuarios hay en total?",
    "¿Cuál es el promedio de años de experiencia?"
]

# Procesar cada pregunta
for question in questions:
    print("\n" + "="*50)
    print(f"Pregunta: {question}")
    print("-"*50)
    response = query_chain.process_question(question)
    print(f"Respuesta: {response}")

2024-11-16 17:43:38,337 - INFO - Procesando pregunta: ¿Cuáles son los últimos 5 usuarios registrados?



Pregunta: ¿Cuáles son los últimos 5 usuarios registrados?
--------------------------------------------------


/var/folders/rn/1q7mj00d6b3fwpkj8tndfp8c0000gn/T/ipykernel_18777/1573036840.py:87: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  query = self.llm.predict(prompt_value.to_string())
2024-11-16 17:43:39,799 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-16 17:43:39,811 - INFO - Consulta SQL generada: SELECT * FROM usuarios ORDER BY fecha_registro DESC LIMIT 5;
2024-11-16 17:43:39,816 - INFO - Resultado de la consulta: [(1, 'Juan Pérez', 'Licenciatura', 'Seguridad de Redes', 5, 'juan.perez@email.com', '123456789', 'Madrid', datetime.date(2024, 11, 16), 'Especialista en firewalls y gestión de redes seguras.'), (2, 'María López', 'Máster', 'Criptografía', 3, 'maria.lopez@email.com', '987654321', 'Barcelona', datetime.date(2024, 11, 16), 'Experta en algoritmos de cifrado y técnicas de protección de datos.'), (3, 'Carlos García', 

Respuesta: Los últimos 5 usuarios registrados son:

1. **Juan Pérez**  
   - **Título:** Licenciatura  
   - **Especialidad:** Seguridad de Redes  
   - **Email:** juan.perez@email.com  
   - **Teléfono:** 123456789  
   - **Ubicación:** Madrid  
   - **Descripción:** Especialista en firewalls y gestión de redes seguras.  
   - **Fecha de registro:** 16 de noviembre de 2024  

2. **María López**  
   - **Título:** Máster  
   - **Especialidad:** Criptografía  
   - **Email:** maria.lopez@email.com  
   - **Teléfono:** 987654321  
   - **Ubicación:** Barcelona  
   - **Descripción:** Experta en algoritmos de cifrado y técnicas de protección de datos.  
   - **Fecha de registro:** 16 de noviembre de 2024  

3. **Carlos García**  
   - **Título:** Doctorado  
   - **Especialidad:** Análisis de Vulnerabilidades  
   - **Email:** carlos.garcia@email.com  
   - **Teléfono:** 654123987  
   - **Ubicación:** Valencia  
   - **Descripción:** Lideró proyectos de investigación sobre vulnerabilida

2024-11-16 17:43:47,144 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-16 17:43:47,146 - INFO - Consulta SQL generada: SELECT COUNT(*) FROM usuarios;
2024-11-16 17:43:47,152 - INFO - Resultado de la consulta: [(5,)]
2024-11-16 17:43:47,956 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-16 17:43:47,958 - INFO - Respuesta generada exitosamente
2024-11-16 17:43:47,958 - INFO - Procesando pregunta: ¿Cuál es el promedio de años de experiencia?


Respuesta: Hay un total de 5 usuarios registrados en la base de datos.

Pregunta: ¿Cuál es el promedio de años de experiencia?
--------------------------------------------------


2024-11-16 17:43:48,877 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-16 17:43:48,880 - INFO - Consulta SQL generada: SELECT AVG(experiencia_años) AS promedio_experiencia FROM usuarios;
2024-11-16 17:43:48,886 - INFO - Resultado de la consulta: [(Decimal('4.8000'),)]
2024-11-16 17:43:49,912 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-16 17:43:49,916 - INFO - Respuesta generada exitosamente


Respuesta: El promedio de años de experiencia es de 4.8 años. Esto indica que, en general, los usuarios tienen una experiencia considerable en su campo.
